In [32]:
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import zipfile
from tqdm import tqdm
import base64
import pathlib

In [4]:
load_dotenv()

True

In [5]:
client = OpenAI()

In [2]:
path = './pliki_z_fabryki'
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [3]:
files

['2024-11-12_report-00-sektor_C4.txt',
 '2024-11-12_report-01-sektor_A1.txt',
 '2024-11-12_report-02-sektor_A3.txt',
 '2024-11-12_report-03-sektor_A3.txt',
 '2024-11-12_report-04-sektor_B2.txt',
 '2024-11-12_report-05-sektor_C1.txt',
 '2024-11-12_report-06-sektor_C2.txt',
 '2024-11-12_report-07-sektor_C4.txt',
 '2024-11-12_report-08-sektor_A1.txt',
 '2024-11-12_report-09-sektor_C2.txt',
 '2024-11-12_report-10-sektor-C1.mp3',
 '2024-11-12_report-11-sektor-C2.mp3',
 '2024-11-12_report-12-sektor_A1.mp3',
 '2024-11-12_report-13.png',
 '2024-11-12_report-14.png',
 '2024-11-12_report-15.png',
 '2024-11-12_report-16.png',
 '2024-11-12_report-17.png',
 '2024-11-12_report-99',
 'weapons_tests.zip']

In [9]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [12]:
base64_image = encode_image('pliki_z_fabryki/2024-11-12_report-13.png')

In [47]:
def ask_image(image_path):
  base64_image = encode_image(image_path) 
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Wydobądź dla nas proszę tylko notatki zawierające informacje o schwytanych ludziach \
              lub o śladach ich obecności oraz o naprawionych usterkach hardwarowych (pomiń te związane z softem oraz pomiń katalog z faktami). \
              Odpowiedz tylko jednym słowem - Ludzie lub Urządzenia, jeśli notatka zawiera odpowiednie informacje. W przeciwnym wypadku odpowiedź - Pomiń."
          },
          {
            "type": "image_url",
            "image_url": {
              "url":  f"data:image/jpeg;base64,{base64_image}"
            },
          },
        ],
      }
    ],
  )
  answer = response.choices[0].message.content
  return answer

In [48]:
def ask_text(txt_path):
    with open(txt_path, 'r') as f:
        data = f.read()
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": data},
            {"role": "user", "content": "Wydobądź dla nas proszę tylko notatki zawierające informacje o schwytanych ludziach \
              lub o śladach ich obecności oraz o naprawionych usterkach hardwarowych (pomiń te związane z softem oraz pomiń katalog z faktami). \
              Odpowiedz tylko jednym słowem - Ludzie lub Urządzenia, jeśli notatka zawiera odpowiednie informacje. W przeciwnym wypadku odpowiedź - Pomiń."}
        ]
    )
    answer = response.choices[0].message.content
    return answer

In [49]:
def ask_audio(audio_path):
    with open(audio_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1", 
            file=audio_file
            )
    response = client.chat.completions.create(
    model="gpt-4o",
        messages=[
            {"role": "system", "content": transcription.text},
            {"role": "user", "content": "Wydobądź dla nas proszę tylko notatki zawierające informacje o schwytanych ludziach \
              lub o śladach ich obecności oraz o naprawionych usterkach hardwarowych (pomiń te związane z softem oraz pomiń katalog z faktami). \
              Odpowiedz tylko jednym słowem - Ludzie lub Urządzenia, jeśli notatka zawiera odpowiednie informacje. W przeciwnym wypadku odpowiedź - Pomiń."}
        ]
    )
    answer = response.choices[0].message.content
    return answer

In [44]:
ask_audio('pliki_z_fabryki/2024-11-12_report-10-sektor-C1.mp3')

'Ludzie'

In [50]:
def read_content(folder_path):
    people = []
    hardware = []
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for file in tqdm(files):
        file_path = os.path.join(folder_path, file)
        if pathlib.Path(file_path).suffix == '.png':
            answer = ask_image(file_path)
        elif pathlib.Path(file_path).suffix == '.txt':
            answer = ask_text(file_path)
        elif pathlib.Path(file_path).suffix == '.mp3':
            answer = ask_audio(file_path)
        if answer.lower() == 'ludzie':
            people.append(file)
        elif answer.lower() == 'urządzenia':
            hardware.append(file)
        else:
            print(f'File: {file} skipped')
    return people, hardware

In [51]:
people, hardware = read_content('pliki_z_fabryki')

 10%|█         | 2/20 [00:01<00:10,  1.68it/s]

File: 2024-11-12_report-01-sektor_A1.txt skipped


 15%|█▌        | 3/20 [00:01<00:09,  1.72it/s]

File: 2024-11-12_report-02-sektor_A3.txt skipped


 20%|██        | 4/20 [00:02<00:09,  1.75it/s]

...

File: 2024-11-12_report-14.png skipped


 85%|████████▌ | 17/20 [00:27<00:09,  3.02s/it]

File: 2024-11-12_report-16.png skipped


100%|██████████| 20/20 [00:30<00:00,  1.52s/it]

File: 2024-11-12_report-17.png skipped
File: 2024-11-12_report-99 skipped
File: weapons_tests.zip skipped


In [64]:
people

['2024-11-12_report-00-sektor_C4.txt',
 '2024-11-12_report-07-sektor_C4.txt',
 '2024-11-12_report-10-sektor-C1.mp3']

In [63]:
hardware

['2024-11-12_report-13.png', '2024-11-12_report-15.png']

In [79]:
solution_response = {
   "task": "kategorie",
   "apikey": os.getenv('AI_DEVS_3_API_KEY'),
   "answer": {
       "people": people,
       "hardware": ['2024-11-12_report-13.png', '2024-11-12_report-15.png', '2024-11-12_report-17.png'],
   }
}

In [80]:
response = requests.post('https://centrala.ag3nts.org/report', json=solution_response)

In [81]:
response

<Response [200]>

In [82]:
response.json()

{'code': 0, 'message': '{{FLG:KLASYFIKACJA}}'}